In [1]:
import cudf
import dask_cudf

In [2]:
import dask_cudf

In [3]:
from dask_cuml.linear_regression import LinearRegression

In order to run this notebook, you will first need to run a dask scheduler and number of dask workers:
- Run a dask scheduler with:  ```dask-scheduler --scheduler-file=cluster.json```
- Run N dask workers with:  ```mpirun -np N dask-mpi --no-nanny --nthreads 10 --no-scheduler --scheduler-file cluster.json```

In [4]:
# from dask.distributed import Client
# from dask_cuda import LocalCUDACluster
# cluster = LocalCUDACluster(threads_per_worker = 10)


In [5]:
from dask.distributed import Client
client = Client(scheduler_file = "../cluster.json")
client

Client Scheduler: tcp://10.2.169.115:8786 Dashboard: http://10.2.169.115:8787/status,Cluster Workers: 2 Cores: 2 Memory: 3.15 GB


In [6]:
X = cudf.DataFrame([('a', [0, 1, 2, 3, 4])])
y = cudf.Series([0, 1, 2, 3, 4])


In [7]:
X_df = dask_cudf.from_cudf(X, chunksize=1).persist()
y_df = dask_cudf.from_cudf(y, chunksize=1).persist()

In [8]:
client.who_has()

{"('from_cudf-8f179dedebc84dbea0a79cb2905be8f9', 0)": (),
 "('from_cudf-8f179dedebc84dbea0a79cb2905be8f9', 1)": (),
 "('from_cudf-8f179dedebc84dbea0a79cb2905be8f9', 2)": (),
 "('from_cudf-8f179dedebc84dbea0a79cb2905be8f9', 3)": (),
 "('from_cudf-fc3c050cbd3b42928497b52daf3ffd4b', 0)": (),
 "('from_cudf-fc3c050cbd3b42928497b52daf3ffd4b', 1)": (),
 "('from_cudf-fc3c050cbd3b42928497b52daf3ffd4b', 2)": (),
 "('from_cudf-fc3c050cbd3b42928497b52daf3ffd4b', 3)": ()}

In [9]:
import numba.cuda
import cuml
def print_device(arr):
    print(str(numba.cuda.get_current_device()))
    print(str(cuml.device_of_ptr(arr.as_gpu_matrix(order="F"))))
    
[client.submit(print_device, part) for part in X_df.to_delayed()]

[<Future: status: pending, key: print_device-200b5ef9ae81e92ef6d4892deda0b1d9>,
 <Future: status: pending, key: print_device-a0f1ea2e85097629df8fac9a4b5b0130>,
 <Future: status: pending, key: print_device-cf7c4344adb8b0c1adc646991c2909af>,
 <Future: status: pending, key: print_device-4454b6d7cf74fc38896a7625bff9ca16>]

Set each worker to host dfs on a different device

In [ ]:
import numba.cuda

devs = [i.id for i in numba.cuda.cudadrv.devices.gpus]
workers = list(client.has_what().keys())
worker_devs = workers[0:min(len(devs), len(workers))]

In [ ]:
def set_visible(i, n):
    import os
    all_devices = list(range(n))
    vd = ",".join(map(str, all_devices[i:] + all_devices[:i])) 
    print(str(vd))
    os.environ["CUDA_VISIBLE_DEVICES"] = vd

dev_assigned = [client.submit(set_visible, dev, len(devs), workers = [worker]) for dev, worker in zip(devs, worker_devs)]

In [22]:
client.who_has()

{"('from_cudf-69022de2a13245c59bf17af92dd8d3a6', 0)": ('tcp://10.2.169.115:44275',),
 "('from_cudf-69022de2a13245c59bf17af92dd8d3a6', 1)": ('tcp://10.2.169.115:45706',),
 "('from_cudf-69022de2a13245c59bf17af92dd8d3a6', 2)": ('tcp://10.2.169.115:44275',),
 "('from_cudf-69022de2a13245c59bf17af92dd8d3a6', 3)": ('tcp://10.2.169.115:45706',),
 "('from_cudf-935f43ceb3e3463bafbbca6e2a33d221', 0)": ('tcp://10.2.169.115:44275',),
 "('from_cudf-935f43ceb3e3463bafbbca6e2a33d221', 1)": ('tcp://10.2.169.115:45706',),
 "('from_cudf-935f43ceb3e3463bafbbca6e2a33d221', 2)": ('tcp://10.2.169.115:44275',),
 "('from_cudf-935f43ceb3e3463bafbbca6e2a33d221', 3)": ('tcp://10.2.169.115:45706',),
 'print_device-0af762345947f1f3beba182fc435e7c6': ('tcp://10.2.169.115:45706',),
 'print_device-214063a90498a67de2855b85bbef57fc': ('tcp://10.2.169.115:44275',),
 'print_device-85c539a30774eb73e23e65689c6bb7cb': ('tcp://10.2.169.115:45706',),
 'print_device-ce76cbbf33de3d90a1f3675f75c08c31': ('tcp://10.2.169.115:44275'

In [9]:
lr = LinearRegression()

In [10]:
res = lr.fit(X_df, y_df)

Co-locating data!
parts=[<Future: status: finished, type: tuple, key: tuple-14ec3974-b89a-47fe-a9c4-44c1e667b1d3>, <Future: status: finished, type: tuple, key: tuple-7d45e68a-0d62-47ff-b0bc-fb4c8b2820ee>, <Future: status: finished, type: tuple, key: tuple-dabbd087-c904-4173-b80e-e31cc65f323f>, <Future: status: finished, type: tuple, key: tuple-b487e702-3119-4cd2-afa0-476ecd1c77a9>]
key_to_part_dict={'tuple-dabbd087-c904-4173-b80e-e31cc65f323f': <Future: status: finished, type: tuple, key: tuple-dabbd087-c904-4173-b80e-e31cc65f323f>, 'tuple-7d45e68a-0d62-47ff-b0bc-fb4c8b2820ee': <Future: status: finished, type: tuple, key: tuple-7d45e68a-0d62-47ff-b0bc-fb4c8b2820ee>, 'tuple-14ec3974-b89a-47fe-a9c4-44c1e667b1d3': <Future: status: finished, type: tuple, key: tuple-14ec3974-b89a-47fe-a9c4-44c1e667b1d3>, 'tuple-b487e702-3119-4cd2-afa0-476ecd1c77a9': <Future: status: finished, type: tuple, key: tuple-b487e702-3119-4cd2-afa0-476ecd1c77a9>}
[(('10.2.169.115', 42714), <Future: status: finished,


res

In [12]:
lr.coeffs

[(('10.2.169.115', 44795),
  <Future: status: finished, type: str, key: get_result-cf433f452c20879e2e4bf2cb9c874710>),
 (('10.2.169.115', 41001),
  <Future: status: finished, type: str, key: get_result-10a9f8b3ed7af1948c52e2e0cff7fdcc>)]

In [11]:
g = lr.predict(X_df)

WHO HAS: {'get_result-4d5399cafd8455a7a3cba0a546815e9f': ('tcp://10.2.169.115:34832',), 'get_result-9ff61fe739b69dbe9e9220fb3c283339': ('tcp://10.2.169.115:37956',)}
{'get_result-4d5399cafd8455a7a3cba0a546815e9f': <Future: status: finished, type: Series, key: get_result-4d5399cafd8455a7a3cba0a546815e9f>, 'get_result-9ff61fe739b69dbe9e9220fb3c283339': <Future: status: finished, type: Series, key: get_result-9ff61fe739b69dbe9e9220fb3c283339>}
get_result-4d5399cafd8455a7a3cba0a546815e9f
get_result-9ff61fe739b69dbe9e9220fb3c283339
[(('10.2.169.115', 34832), <Future: status: pending, key: to_gpu_array-4d6f56b55b7a28f8b73ed6f64a6a1e76>), (('10.2.169.115', 37956), <Future: status: pending, key: to_gpu_array-521596a76a504299fbbc3580b29432ed>)]
WORKER_PARTS: {('10.2.169.115', 37956): [Delayed(('from_cudf-d28f756998e640c5aba91650f4b700ef', 3)), Delayed(('from_cudf-d28f756998e640c5aba91650f4b700ef', 1))], ('10.2.169.115', 34832): [Delayed(('from_cudf-d28f756998e640c5aba91650f4b700ef', 2)), Delaye

In [12]:
g

tornado.gen.Return([<Future: status: finished, type: Series, key: get_result-3a3fd34d29b1f359d26f92c95b4d6d24>,
                    <Future: status: finished, type: Series, key: get_result-6b35a1b97185dcd1cfa72110d91e125c>])